In [1]:
import os
import sys
import shutil
from pathlib import Path
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import cv2
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

In [2]:
src = '/home/aditya/yolov3/data/label_kitti/training/label_2/'
dst = '/home/aditya/yolov3/master/data/labels_kitti'
src_files = os.listdir(src)
for file_name in src_files:
    full_file_name = os.path.join(src, file_name)
    if os.path.isfile(full_file_name):
        shutil.copy(full_file_name, dst)

In [3]:
filepath = Path('/home/aditya/yolov3/master/data/labels_kitti/')

mapping = { 'Car ':'Vehicle ', 'Truck ':'Vehicle ', 'Van ':'Vehicle ','Tram ':'Vehicle ', 'Misc ':'Non-Vehicle ', 'Cyclist ':'Non-Vehicle ','Pedestrian ':'Non-Vehicle ','Person_sitting ':'Non-Vehicle ','DontCare ' : 'Non-Vehicle ' }

max_iterations = 7481

In [4]:
filepath

PosixPath('/home/aditya/yolov3/master/data/labels_kitti')

In [5]:
for x in list(filepath.glob('*.txt'))[:max_iterations]:
    # Read in the file
    with open(x, 'r') as file :
      filedata = file.read()
    # Replace the target string
    for key, value in mapping.items():
      filedata = filedata.replace(key,value)
    # Write the file out again
    with open(x, 'w') as file:
      file.write(filedata)

In [7]:
#set your data_set absolute path
# test example
kitti_img_path ='/home/aditya/yolov3/master/data/images/'
kitti_label_path = '/home/aditya/yolov3/master/data/labels_kitti/'


#transformed lables to save path
kitti_label_tosave_path = '/home/aditya/yolov3/master/data/labels/'

index = 0
cvfont = cv2.FONT_HERSHEY_SIMPLEX

kitti_names = open('data/kitti1.names','r')
kitti_names_contents = kitti_names.readlines()                
kitti_images = os.listdir(kitti_img_path)
kitti_labels = os.listdir(kitti_label_path)

kitti_images.sort()
kitti_labels.sort()

kitti_names_dic_key = []
for class_name in kitti_names_contents:
    kitti_names_dic_key.append(class_name.rstrip())
values = range(len(kitti_names_dic_key))
kitti_names_num = dict(zip(kitti_names_dic_key,values))

#print(kitti_names_num)
#os.makedirs("imagewbox", exist_ok=True)


indexi = 1

for indexi in range(len(kitti_images)):
    kitti_img_totest_path = kitti_img_path + kitti_images[indexi]
    kitti_label_totest_path = kitti_label_path + kitti_labels[indexi]
    #print(kitti_label_totest_path,kitti_img_totest_path)
    
    kitti_img_totest = cv2.imread(kitti_img_totest_path)
    #print(kitti_img_totest,type(kitti_img_totest))
    img_height, img_width = kitti_img_totest.shape[0],kitti_img_totest.shape[1]
    
    #plotting image
    fig,ax = plt.subplots(1)
    ax.imshow(kitti_img_totest)
    
    kitti_label_totest = open(kitti_label_totest_path,'r')
    
    label_contents = kitti_label_totest.readlines()
    #print(label_contents)
    
    real_label_to_write = []
    for line in label_contents:
        data = line.split(' ')
        x=y=w=h=0
        if(len(data) == 15):
            class_str = data[0]
            if(class_str != 'Non-Vehicle'):
                # for kitti calls is a string
                # trans this to number by using kitti.names
                #(x,y) center (w,h) size
                
                x1 = float(data[4])
                y1 = float(data[5])
                x2 = float(data[6])
                y2 = float(data[7])
                w = x2-x1
                h = y2-y1
                
                #print("Bounding Box:", class_str,x1,y1,w,h)
                rect = patches.Rectangle((x1,y1),w,h,linewidth=1,edgecolor='r',facecolor='none')
                ax.add_patch(rect)
                intx1 = int(x1)
                inty1 = int(y1)
                intx2 = int(x2)
                inty2 = int(y2)

                bbox_center_x = float( (x1 + (x2 - x1) / 2.0) / img_width)
                bbox_center_y = float( (y1 + (y2 - y1) / 2.0) / img_height)
                bbox_width = float((x2 - x1) / img_width)
                bbox_height = float((y2 - y1) / img_height)
                
                #cv2.rectangle(kitti_img_totest, (intx1,inty1), (intx2,inty2), (0,255,0), 2)
                line_to_write = str(kitti_names_num[class_str]) + ' ' + str(bbox_center_x)+ ' ' + str(bbox_center_y)+ ' ' + str(bbox_width)+ ' ' + str(bbox_height) +'\n'
                #print("Transformed Label:", line_to_write)
                real_label_to_write.append(line_to_write)
                sys.stdout.write(str(int((indexi/len(kitti_images))*100))+'% '+'*******************->' "\r" )
                sys.stdout.flush()
                #plt.show()
    
    if len(real_label_to_write) > 0:
        real_label = open(kitti_label_tosave_path + kitti_labels[indexi],'w')
        for line in real_label_to_write:
            real_label.write(line)
        real_label.close()
    #plt.show()
    #plt.savefig(f"imagewbox/{kitti_images[indexi]}", bbox_inches="tight", pad_inches=0.0)
    plt.close()
kitti_names.close()

print("Labels tranfrom finished!")

Labels tranfrom finished!


In [42]:
kitti_labels = os.listdir(kitti_label_tosave_path)


f = open('pathtofile.txt','w')
for item in kitti_labels:
    if item.split('.')[1] == 'txt':
        img = item.split('.')[0]+".png"
        f.write(kitti_img_path+img+'\n')
f.close()

In [43]:
pathtofile = pd.read_csv('/home/aditya/yolov3/master/pathtofile.txt',sep='\n',header=None)

In [45]:
pathtofile.head()

,0
0,/home/aditya/yolov3/master/data/images/003436.png
1,/home/aditya/yolov3/master/data/images/006475.png
2,/home/aditya/yolov3/master/data/images/002572.png
3,/home/aditya/yolov3/master/data/images/002523.png
4,/home/aditya/yolov3/master/data/images/001977.png


In [46]:
len(pathtofile)

6820

In [47]:
#train-test set split

img_train, img_test = train_test_split(pathtofile, test_size = 0.1, random_state=42)

In [48]:
len(img_train),len(img_test)

(6138, 682)

In [49]:
#train-validation set split

img_train1, img_valid = train_test_split(img_train, test_size = 0.055, random_state=42)

In [50]:
len(img_train1),len(img_valid)

(5800, 338)

In [51]:
img_train1.to_csv('/home/aditya/yolov3/master/data/train.txt',sep='\n',header=None, index=False)

In [52]:
img_test.to_csv('/home/aditya/yolov3/master/data/test.txt',sep='\n',header=None, index=False)

In [53]:
img_valid.to_csv('/home/aditya/yolov3/master/data/valid.txt',sep='\n',header=None, index=False)

In [54]:
trainsetpath = pd.read_csv('/home/aditya/yolov3/master/data/train.txt',sep='\n',header=None)

In [55]:
len(trainsetpath[0])

5800

In [56]:
#copying file to new folder for generating output for pseudo-labelling

dst_dir = "/home/aditya/yolov3/master/data/pseudo/images/"
pathlist = trainsetpath[0]

In [57]:
#pathlist

for path in pathlist:
    src_dir = Path(path)
    shutil.copy(src_dir,dst_dir) 